# Using POST

POST allows you to run a query with multiple inputs at once. The output will be a dictionary of dictionaries.

In [1]:
import requests, sys
from pprint import pprint

server = "http://rest.ensembl.org"
ext = "/lookup/id"
headers={ "Content-Type" : "application/json", "Accept" : "application/json"}
r = requests.post(server+ext, headers=headers, data='{ "ids" : ["ENSG00000157764", "ENSG00000248378" ] }')

# error checking removed for space
 
decoded = r.json()
pprint (decoded)

{'ENSG00000157764': {'assembly_name': 'GRCh38',
                     'biotype': 'protein_coding',
                     'db_type': 'core',
                     'description': 'B-Raf proto-oncogene, serine/threonine '
                                    'kinase [Source:HGNC Symbol;Acc:HGNC:1097]',
                     'display_name': 'BRAF',
                     'end': 140924929,
                     'id': 'ENSG00000157764',
                     'logic_name': 'ensembl_havana_gene_homo_sapiens',
                     'object_type': 'Gene',
                     'seq_region_name': '7',
                     'source': 'ensembl_havana',
                     'species': 'homo_sapiens',
                     'start': 140719327,
                     'strand': -1,
                     'version': 14},
 'ENSG00000248378': {'assembly_name': 'GRCh38',
                     'biotype': 'lncRNA',
                     'db_type': 'core',
                     'description': 'novel transcript',
                 

There is a helper function in POST. You can specify both helper functions in your script and use whichever one you need.

In [3]:
def fetch_endpoint_POST(server, request, data, content_type='application/json'):

    r = requests.post(server+request,
                      headers={ "Accept" : content_type},
                      data=data )

    if not r.ok:
        r.raise_for_status()
        sys.exit()

    if content_type == 'application/json':
        return r.json()
    else:
        return r.text

### Optional parameters

In order to add optional parameters to your POST query, you can just add them onto the extention with a slash. For example if you wanted to mask UTRs when running the [sequence_id_post](http://rest.ensembl.org/documentation/info/sequence_id_post) endpoint, you could specify your extension as:

        `ext = "sequence/id/mask_feature=1"`


### Input

Your input list for POST queries need to be a JSON list. You can create this from a list in Python using the [JSON module](https://docs.python.org/3/library/json.html):

        `data = json.dumps({ "ids" : my_list })`

### Output

The Output from POST queries will be a dictionary of dictionaries. To access items, you could use your input list as your keys, or you could move through the dictionary with:

		`for key, value in post_query.items():`

## Example

The following scripts inputs a list of variants in HGVS format into the VEP and gets out the IDs of known colocated variants, including failed variants (an optional parameter):

In [6]:
import requests, sys, json
from pprint import pprint

def fetch_endpoint(server, request, content_type):

    r = requests.get(server+request, headers={ "Accept" : content_type})

    if not r.ok:
        r.raise_for_status()
        sys.exit()

    if content_type == 'application/json':
        return r.json()
    else:
        return r.text

def fetch_endpoint_POST(server, request, data, content_type):

    r = requests.post(server+request,
                      headers={ "Accept" : content_type},
                      data=data )

    if not r.ok:
        r.raise_for_status()
        sys.exit()

    if content_type == 'application/json':
        return r.json()
    else:
        return r.text

# define the server, extension and content type
server = "http://rest.ensembl.org/"
con = "application/json"
vep_ext = "vep/homo_sapiens/hgvs/failed=1"

# create the list of HGVS annotations
hgvs = ["ENST00000366667.4:c.803T>C", "ENST00000335295.4:c.20A>T", "ENST00000415952.1:c.-149-34206G>T"]

# convert the list into json format
hgvs_json = json.dumps({ "hgvs_notations" : hgvs })
print(hgvs_json)

# run the query
post_vep = fetch_endpoint_POST(server, vep_ext, hgvs_json, con)

# move through the results
for variant in post_vep:
    
    # get the data
    input = variant['input']
    colocated_list = []
    for colocated in variant['colocated_variants']:
        colocated_list.append(colocated['id'])  
    print (input + ": " + (', '.join(colocated_list)))

{"hgvs_notations": ["ENST00000366667.4:c.803T>C", "ENST00000335295.4:c.20A>T", "ENST00000415952.1:c.-149-34206G>T"]}
ENST00000366667.4:c.803T>C: CM920010, rs699
ENST00000335295.4:c.20A>T: CD830010, CM097155, CM880038, rs334
ENST00000415952.1:c.-149-34206G>T: rs123


# Exercises 6

1\. Fetch the all the transcripts of *ESPN* using the lookup endpoint. Fetch the cDNA sequences of all transcripts using a single POST request, and print in FASTA format.

In [12]:
# Exercise 6.1
import requests, sys, json
from pprint import pprint

def fetch_endpoint(server, request, content_type):

    r = requests.get(server+request, headers={ "Accept" : content_type})

    if not r.ok:
        r.raise_for_status()
        sys.exit()

    if content_type == 'application/json':
        return r.json()
    else:
        return r.text

# Get the gene name from the command line
gene_name = "ESPN"

# define the general URL parameters
server = "http://rest.ensembl.org/"
con = "application/json"
ext_get_lookup = "lookup/symbol/homo_sapiens/" + gene_name + "?" + "expand=1"

# submit the query
get_lookup = fetch_endpoint(server, ext_get_lookup, con)

pprint (get_lookup['Transcript'])



[{'Exon': [{'assembly_name': 'GRCh38',
            'db_type': 'core',
            'end': 6460980,
            'id': 'ENSE00001819746',
            'object_type': 'Exon',
            'seq_region_name': '1',
            'species': 'homo_sapiens',
            'start': 6460508,
            'strand': 1,
            'version': 1},
           {'assembly_name': 'GRCh38',
            'db_type': 'core',
            'end': 6461367,
            'id': 'ENSE00003489552',
            'object_type': 'Exon',
            'seq_region_name': '1',
            'species': 'homo_sapiens',
            'start': 6461180,
            'strand': 1,
            'version': 2}],
  'Parent': 'ENSG00000187017',
  'assembly_name': 'GRCh38',
  'biotype': 'processed_transcript',
  'db_type': 'core',
  'display_name': 'ESPN-205',
  'end': 6461367,
  'id': 'ENST00000468561',
  'is_canonical': 0,
  'logic_name': 'havana_homo_sapiens',
  'object_type': 'Transcript',
  'seq_region_name': '1',
  'source': 'havana',
  'species': 

  'Parent': 'ENSG00000187017',
  'Translation': {'Parent': 'ENST00000636330',
                  'db_type': 'core',
                  'end': 6456197,
                  'id': 'ENSP00000490186',
                  'length': 1362,
                  'object_type': 'Translation',
                  'species': 'homo_sapiens',
                  'start': 6424956},
  'assembly_name': 'GRCh38',
  'biotype': 'protein_coding',
  'db_type': 'core',
  'display_name': 'ESPN-215',
  'end': 6456671,
  'id': 'ENST00000636330',
  'is_canonical': 0,
  'logic_name': 'havana_homo_sapiens',
  'object_type': 'Transcript',
  'seq_region_name': '1',
  'source': 'havana',
  'species': 'homo_sapiens',
  'start': 6424788,
  'strand': 1,
  'version': 1},
 {'Exon': [{'assembly_name': 'GRCh38',
            'db_type': 'core',
            'end': 6425249,
            'id': 'ENSE00003825592',
            'object_type': 'Exon',
            'seq_region_name': '1',
            'species': 'homo_sapiens',
            'start': 64

2\. You have the following list of variants:
```rs1415919662, rs957333053, rs762944488, rs1372123943, rs553810871, rs1451237599, rs751376931```
Get the variant class, evidence attributes, source and the most_severe_consequence for all variants using the variation POST endpoint.

In [ ]:
# Exercise 6.2



[Next page: Exercises 6 – answers](6_Using_POST_answers.ipynb)